In [82]:
from google.cloud import bigquery
import pandas  as pd
from urllib.error import HTTPError

In [1]:
airports = pd.read_csv('dataset/csvs/AirPorts.csv')
flights =  pd.read_csv('dataset/csvs/Flights (2).csv')

In [2]:
airports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   airport  397 non-null    object
 1   iata     397 non-null    object
 2   icao     326 non-null    object
 3   city     395 non-null    object
 4   state    395 non-null    object
 5   country  397 non-null    object
dtypes: object(6)
memory usage: 18.7+ KB


In [3]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 5 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   Flight  object
 1   Orgin   object
 2   Dept    object
 3   Stops   object
 4   Price   int64 
dtypes: int64(1), object(4)
memory usage: 381.5+ MB


In [4]:
print(f"pre: missing count: {flights['Price'].isna().sum()}")


pre: missing count: 0


In [5]:
flights.head(5)

,Flight,Orgin,Dept,Stops,Price
0,Tianjin Airlines,SVG,STN,2,305
1,LATAM Airlines,GDL,DUB,3,341
2,Qatar Airways,YVR,NGO,non-stop,454
3,Tianjin Airlines,BWI,PTY,non-stop,878
4,Shandong Airlines,MDQ,MTR,non-stop,878


In [6]:
flights['Orgin'].value_counts()

Orgin
BCN    25627
SAW    25602
FOR    25585
OMA    25571
CLE    25567
       ...  
JED    24840
VCP    24839
YYC    24823
NGB    24782
TXL    24759
Name: count, Length: 397, dtype: int64

In [95]:
def check_bq_table(project_id, dataset_id, table_id):
    dataset_ref = bigquery.DatasetReference(project_id, dataset_id)
    table_ref = bigquery.TableReference(dataset_ref, table_id)
    try:
        client.get_table(table_ref)
        return True
    except :
        return False

In [96]:
def create_bq_table(project_id , dataset_id , table_id) -> bool :

    if check_bq_table(project_id, dataset_id , table_id) :
        return False
        
    client = bigquery.Client()  # Construct a BigQuery client object.
    
    types_mapping = {'object': 'STRING', 'string': 'STRING' ,'int64':'INTEGER' }
    
    schema = []
    
    for column in flights.columns: # Creating Schema Dynamically from the dataframe columns
        column_type = str(flights[column].dtype)
        bq_type = types_mapping[column_type]
        schema.append(bigquery.SchemaField(column, bq_type, mode="REQUIRED"))
        
    table = bigquery.Table(f"{project_id}.{dataset_id}.{table_id}", schema=schema)
    table = client.create_table(table)  # Make an API request.
    print(
        "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
    )
    return True

In [97]:
create_bq_table('resolute-choir-403411', 'flights','test')

Created table resolute-choir-403411.flights.test


True

In [103]:
client.project

'resolute-choir-403411'

In [14]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 5 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   Flight  object
 1   Orgin   object
 2   Dept    object
 3   Stops   object
 4   Price   int64 
dtypes: int64(1), object(4)
memory usage: 381.5+ MB


In [12]:
airports['iata'] = airports['iata'].astype('string')

In [13]:
airports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   airport  397 non-null    string
 1   iata     397 non-null    string
 2   icao     326 non-null    object
 3   city     395 non-null    object
 4   state    395 non-null    object
 5   country  397 non-null    object
dtypes: object(4), string(2)
memory usage: 18.7+ KB


In [104]:
import re


In [107]:
table_id = "flights (2)"

In [108]:
table_id = re.sub('[^A-Za-z0-9]+', '', table_id) 

In [109]:
table_id

'flights2'